In [3]:
import os
import openai

In [23]:
import json

In [27]:
import tenacity

In [31]:
from bs4 import BeautifulSoup
import random

In [4]:
# Load your API key from an environment variable or secret management service
# openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = "<REDACTED>"

## Test

In [13]:
test_chat_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

test_chat_completion

<OpenAIObject chat.completion id=chatcmpl-7yZbRDyFfovMod5net0UvJaQkTV6j at 0x7f44281715b0> JSON: {
  "id": "chatcmpl-7yZbRDyFfovMod5net0UvJaQkTV6j",
  "object": "chat.completion",
  "created": 1694670893,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The 2020 World Series was played at the Globe Life Field in Arlington, Texas."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 53,
    "completion_tokens": 18,
    "total_tokens": 71
  }
}

## Survey content

In [10]:
survey_text = """
<h1>How do you think young adults today compare with their parents' generation on each of the following?</h1>
  <p>For the following questions, is this action easier, harder, or the same for young adults today compared to their
    parents' generation?</p>
  <crowd-form><form method="" action="">
    <div id="container">
    <div id="0ad80e72-02e7-4cfc-a298-cc1568f665ff&quot;">
        <div class="QuestionContent">
          <p>Buying a home</p>
        </div>
        <div>
          <input type="radio" id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Easier" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" value="'Easier" required="">
          <label for="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Harder" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" value="'Harder" required="">
          <label for="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Same" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" value="'Same" required="">
          <label for="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Same">About the same</label>
        </div>
      </div><div id="c799476d-db52-4dfe-a5f2-187e3894d280&quot;">
        <div class="QuestionContent">
          <p>Staying in touch with family and friends</p>
        </div>
        <div>
          <input type="radio" id="c799476d-db52-4dfe-a5f2-187e3894d280-'Easier" name="c799476d-db52-4dfe-a5f2-187e3894d280" value="'Easier" required="">
          <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="c799476d-db52-4dfe-a5f2-187e3894d280-'Harder" name="c799476d-db52-4dfe-a5f2-187e3894d280" value="'Harder" required="">
          <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="c799476d-db52-4dfe-a5f2-187e3894d280-'Same" name="c799476d-db52-4dfe-a5f2-187e3894d280" value="'Same" required="">
          <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Same">About the same</label>
        </div>
      </div><div id="40b468aa-4354-4679-a297-068cc7d8f897&quot;">
        <div class="QuestionContent">
          <p>Paying for college</p>
        </div>
        <div>
          <input type="radio" id="40b468aa-4354-4679-a297-068cc7d8f897-'Easier" name="40b468aa-4354-4679-a297-068cc7d8f897" value="'Easier" required="">
          <label for="40b468aa-4354-4679-a297-068cc7d8f897-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="40b468aa-4354-4679-a297-068cc7d8f897-'Harder" name="40b468aa-4354-4679-a297-068cc7d8f897" value="'Harder" required="">
          <label for="40b468aa-4354-4679-a297-068cc7d8f897-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="40b468aa-4354-4679-a297-068cc7d8f897-'Same" name="40b468aa-4354-4679-a297-068cc7d8f897" value="'Same" required="">
          <label for="40b468aa-4354-4679-a297-068cc7d8f897-'Same">About the same</label>
        </div>
      </div><div id="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d&quot;">
        <div class="QuestionContent">
          <p>Finding a job</p>
        </div>
        <div>
          <input type="radio" id="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Easier" name="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d" value="'Easier" required="">
          <label for="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Harder" name="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d" value="'Harder" required="">
          <label for="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Same" name="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d" value="'Same" required="">
          <label for="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Same">About the same</label>
        </div>
      </div><div id="8f511d04-bcba-43a6-ac5f-eccdf5f049af&quot;">
        <div class="QuestionContent">
          <p>Getting into college</p>
        </div>
        <div>
          <input type="radio" id="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Easier" name="8f511d04-bcba-43a6-ac5f-eccdf5f049af" value="'Easier" required="">
          <label for="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Harder" name="8f511d04-bcba-43a6-ac5f-eccdf5f049af" value="'Harder" required="">
          <label for="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Same" name="8f511d04-bcba-43a6-ac5f-eccdf5f049af" value="'Same" required="">
          <label for="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Same">About the same</label>
        </div>
      </div><div id="d73751f3-b779-4351-914b-8605dd5c0bfc&quot;">
        <div class="QuestionContent" style="margin-bottom: -2em;">
          <p>What is your age?</p>
        </div>
        <crowd-input name="d73751f3-b779-4351-914b-8605dd5c0bfc" label="What is your age?" allowed-pattern="^\d*$" error-message="Please enter ONLY numbers" auto-validate="" required="" tabindex="0" aria-disabled="false">
        </crowd-input>
      </div><div id="f73efca0-d7fb-4fb6-9527-884cac8aac41&quot;">
        <div class="QuestionContent">
          <p>Finding a spouse or partner</p>
        </div>
        <div>
          <input type="radio" id="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Easier" name="f73efca0-d7fb-4fb6-9527-884cac8aac41" value="'Easier" required="">
          <label for="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Harder" name="f73efca0-d7fb-4fb6-9527-884cac8aac41" value="'Harder" required="">
          <label for="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Same" name="f73efca0-d7fb-4fb6-9527-884cac8aac41" value="'Same" required="">
          <label for="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Same">About the same</label>
        </div>
      </div><div id="727d4914-85d7-4e5d-8024-0d5af55027ac&quot;">
        <div class="QuestionContent">
          <p>Saving for the future</p>
        </div>
        <div>
          <input type="radio" id="727d4914-85d7-4e5d-8024-0d5af55027ac-'Easier" name="727d4914-85d7-4e5d-8024-0d5af55027ac" value="'Easier" required="">
          <label for="727d4914-85d7-4e5d-8024-0d5af55027ac-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="727d4914-85d7-4e5d-8024-0d5af55027ac-'Harder" name="727d4914-85d7-4e5d-8024-0d5af55027ac" value="'Harder" required="">
          <label for="727d4914-85d7-4e5d-8024-0d5af55027ac-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="727d4914-85d7-4e5d-8024-0d5af55027ac-'Same" name="727d4914-85d7-4e5d-8024-0d5af55027ac" value="'Same" required="">
          <label for="727d4914-85d7-4e5d-8024-0d5af55027ac-'Same">About the same</label>
        </div>
      </div><div id="1b56e8be-127c-47d3-9864-31fceb586234&quot;">
        <div class="QuestionContent">
          <p>Do you currently live in the United States?</p>
        </div>
        <div>
          <input type="radio" id="1b56e8be-127c-47d3-9864-31fceb586234-'Yes" name="1b56e8be-127c-47d3-9864-31fceb586234" value="'Yes" required="">
          <label for="1b56e8be-127c-47d3-9864-31fceb586234-'Yes">Yes</label>
        </div>
        <div>
          <input type="radio" id="1b56e8be-127c-47d3-9864-31fceb586234-'No" name="1b56e8be-127c-47d3-9864-31fceb586234" value="'No" required="">
          <label for="1b56e8be-127c-47d3-9864-31fceb586234-'No">No</label>
        </div>
      </div></div>
  <crowd-button form-action="submit" variant="primary" data-testid="crowd-submit">Submit</crowd-button></form></crowd-form>
  """

### randomize survey order

In [211]:
soup = BeautifulSoup(survey_text, 'html.parser')

# Extract questions
elem_container = soup.find('div', id='container')

# Shuffle questions
questions = elem_container.find_all(recursive=False)
random.shuffle(questions)

# For each question, shuffle the options
for question in questions:
    question_body = question.find_all(recursive=False)

    # shuffle options and not the first div .QuestionContent
    options = question_body[1:]
    random.shuffle(options)
    question_body[1:] = options

    question.extend(question_body)

# Reconstruct the HTML string with shuffled questions and options
shuffled_survey_text = str(soup).replace('\n', '')

# Print the shuffled survey
print(shuffled_survey_text)

<h1>How do you think young adults today compare with their parents' generation on each of the following?</h1><p>For the following questions, is this action easier, harder, or the same for young adults today compared to their    parents' generation?</p><crowd-form><form action="" method=""><div id="container"><div id='0ad80e72-02e7-4cfc-a298-cc1568f665ff"'><div class="QuestionContent"><p>Buying a home</p></div><div><input id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Same" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" required="" type="radio" value="'Same"/><label for="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Same">About the same</label></div><div><input id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Harder" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" required="" type="radio" value="'Harder"/><label for="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Harder">Harder</label></div><div><input id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Easier" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" required="" type="radio" va

#### Playground

In [52]:
soup = BeautifulSoup(survey_text, 'html.parser')

# Extract the questions and options
questions = soup.find_all('div', class_='QuestionContent')
options = soup.find_all('input', type='radio')

# Shuffle the questions
random.shuffle(questions)

# Shuffle the options for each question
for question in questions:
    option_divs = question.find_next_siblings('div')
    random.shuffle(option_divs)

# Reconstruct the HTML string with shuffled questions and options
shuffled_survey_text = str(soup)

# Print the shuffled survey
print(shuffled_survey_text)


<h1>How do you think young adults today compare with their parents' generation on each of the following?</h1>
<p>For the following questions, is this action easier, harder, or the same for young adults today compared to their
    parents' generation?</p>
<crowd-form><form action="" method="">
<div id="container">
<div id='0ad80e72-02e7-4cfc-a298-cc1568f665ff"'>
<div class="QuestionContent">
<p>Buying a home</p>
</div>
<div>
<input id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Easier" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" required="" type="radio" value="'Easier"/>
<label for="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Easier">Easier</label>
</div>
<div>
<input id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Harder" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" required="" type="radio" value="'Harder"/>
<label for="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Harder">Harder</label>
</div>
<div>
<input id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Same" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" required="" 

In [183]:
soup = BeautifulSoup(survey_text, 'html.parser')

elem_container = soup.find('div', id='container')

questions = elem_container.find_all(recursive=False)

random.shuffle(questions)

elem_container.extend(questions)

print(str(soup))


<h1>How do you think young adults today compare with their parents' generation on each of the following?</h1>
<p>For the following questions, is this action easier, harder, or the same for young adults today compared to their
    parents' generation?</p>
<crowd-form><form action="" method="">
<div id="container">
<div id='f73efca0-d7fb-4fb6-9527-884cac8aac41"'>
<div class="QuestionContent">
<p>Finding a spouse or partner</p>
</div>
<div>
<input id="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Easier" name="f73efca0-d7fb-4fb6-9527-884cac8aac41" required="" type="radio" value="'Easier"/>
<label for="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Easier">Easier</label>
</div>
<div>
<input id="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Harder" name="f73efca0-d7fb-4fb6-9527-884cac8aac41" required="" type="radio" value="'Harder"/>
<label for="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Harder">Harder</label>
</div>
<div>
<input id="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Same" name="f73efca0-d7fb-4fb6-9527-884cac8aac41

In [202]:
question_body = questions[2].find_all(recursive=False)
question_body

# divs_without_question_content = soup.find_all('div', class_=lambda x: x != 'QuestionContent')

[<div class="QuestionContent">
 <p>Staying in touch with family and friends</p>
 </div>,
 <div>
 <input id="c799476d-db52-4dfe-a5f2-187e3894d280-'Easier" name="c799476d-db52-4dfe-a5f2-187e3894d280" required="" type="radio" value="'Easier"/>
 <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Easier">Easier</label>
 </div>,
 <div>
 <input id="c799476d-db52-4dfe-a5f2-187e3894d280-'Harder" name="c799476d-db52-4dfe-a5f2-187e3894d280" required="" type="radio" value="'Harder"/>
 <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Harder">Harder</label>
 </div>,
 <div>
 <input id="c799476d-db52-4dfe-a5f2-187e3894d280-'Same" name="c799476d-db52-4dfe-a5f2-187e3894d280" required="" type="radio" value="'Same"/>
 <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Same">About the same</label>
 </div>]

In [203]:
options = question_body[1:]
random.shuffle(options)
question_body[1:] = options

question_body

[<div class="QuestionContent">
 <p>Staying in touch with family and friends</p>
 </div>,
 <div>
 <input id="c799476d-db52-4dfe-a5f2-187e3894d280-'Same" name="c799476d-db52-4dfe-a5f2-187e3894d280" required="" type="radio" value="'Same"/>
 <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Same">About the same</label>
 </div>,
 <div>
 <input id="c799476d-db52-4dfe-a5f2-187e3894d280-'Harder" name="c799476d-db52-4dfe-a5f2-187e3894d280" required="" type="radio" value="'Harder"/>
 <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Harder">Harder</label>
 </div>,
 <div>
 <input id="c799476d-db52-4dfe-a5f2-187e3894d280-'Easier" name="c799476d-db52-4dfe-a5f2-187e3894d280" required="" type="radio" value="'Easier"/>
 <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Easier">Easier</label>
 </div>]

In [204]:
questions[2].extend(question_body)

In [208]:
str(questions[2])

'<div id=\'c799476d-db52-4dfe-a5f2-187e3894d280"\'>\n\n\n\n\n<div class="QuestionContent">\n<p>Staying in touch with family and friends</p>\n</div><div>\n<input id="c799476d-db52-4dfe-a5f2-187e3894d280-\'Same" name="c799476d-db52-4dfe-a5f2-187e3894d280" required="" type="radio" value="\'Same"/>\n<label for="c799476d-db52-4dfe-a5f2-187e3894d280-\'Same">About the same</label>\n</div><div>\n<input id="c799476d-db52-4dfe-a5f2-187e3894d280-\'Harder" name="c799476d-db52-4dfe-a5f2-187e3894d280" required="" type="radio" value="\'Harder"/>\n<label for="c799476d-db52-4dfe-a5f2-187e3894d280-\'Harder">Harder</label>\n</div><div>\n<input id="c799476d-db52-4dfe-a5f2-187e3894d280-\'Easier" name="c799476d-db52-4dfe-a5f2-187e3894d280" required="" type="radio" value="\'Easier"/>\n<label for="c799476d-db52-4dfe-a5f2-187e3894d280-\'Easier">Easier</label>\n</div></div>'

## GPT

In [212]:
instruction = """I want you to act as an average Amazon Mechanical Turk (mturk) survey taker living in the US.
I will provide you with a HTML survey and you will generate a reasonable response to the survey.
You will need to submit the response to the survey as a JSON object, where the keys are the names in the webform, and the values are the values in the webform.
"""

instruction

'I want you to act as an average Amazon Mechanical Turk (mturk) survey taker living in the US.\nI will provide you with a HTML survey and you will generate a reasonable response to the survey.\nYou should respond only with a JSON object to indicate your survey response, where the keys are the names in the webform, and the values are the values in the webform.\n'

In [ ]:
chat_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": instruction},
        {"role": "user", "content": "The HTML survey is given below:\n" + survey_text},
    ]
)

chat_completion

In [ ]:
chat_completion.choices[0]["message"]["content"]

In [ ]:
json.loads(chat_completion.choices[0]["message"]["content"])

## GPT (after shuffling)

In [ ]:
instruction = """I want you to act as an average Amazon Mechanical Turk (mturk) survey taker living in the US.
I will provide you with a HTML survey and you will generate a reasonable response to the survey.
You will need to submit the response to the survey as a JSON object, where the keys are the names in the webform, and the values are the values in the webform.
"""

instruction

In [ ]:
chat_completion_shuffled = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": instruction},
        {"role": "user", "content": "The HTML survey is given below:\n" + shuffled_survey_text},
    ]
)

chat_completion_shuffled

In [ ]:
chat_completion_shuffled.choices[0]["message"]["content"]

In [ ]:
json.loads(chat_completion_shuffled.choices[0]["message"]["content"])

In [ ]:
with open("c.txt", "w") as fp:
    json.dump(chat_completion_shuffled.choices[0]["message"]["content"], fp)

In [ ]:
chat_completion_shuffled.to_dict_recursive()

## GPT4 (shuffled)

In [215]:
instruction = """I want you to act as an average Amazon Mechanical Turk (mturk) survey taker living in the US.
I will provide you with a HTML survey and you will generate a reasonable response to the survey.
You should respond with a JSON object to indicate your survey response, where the keys are the names in the webform, and the values are the values in the webform.
It is VERY IMPORTANT that you respond in valid JSON ONLY. Nothing before or after. Make sure to escape all strings
"""

instruction

'I want you to act as an average Amazon Mechanical Turk (mturk) survey taker living in the US.\nI will provide you with a HTML survey and you will generate a reasonable response to the survey.\nYou should respond with a JSON object to indicate your survey response, where the keys are the names in the webform, and the values are the values in the webform.\nIt is VERY IMPORTANT that you respond in valid JSON ONLY. Nothing before or after. Make sure to escape all strings\n'

In [218]:
sample_survey = """<h1>Food Survey</h1><form action="submit.php" method="post"><h2>Favorite Cuisine:</h2><label><input type="radio" name="favorite_cuisine" value="italian"> Italian</label><label><input type="radio" name="favorite_cuisine" value="mexican"> Mexican</label><label><input type="radio" name="favorite_cuisine" value="indian"> Indian</label><label><input type="radio" name="favorite_cuisine" value="chinese"> Chinese</label><label for="comments">Please share your thoughts:</label><textarea id="comments" name="comments"></textarea><input type="submit" value="Submit"></form>"""
sample_response = """{"favorite_cuisine": "italian","comments": "I love pizza!"}"""

In [224]:
chat_completion_4 = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": instruction},
        {"role": "user", "content": "The HTML survey is given below:\n" + sample_survey},
        {"role": "assistant", "content": sample_response},
        {"role": "user", "content": "The HTML survey is given below:\n" + shuffled_survey_text},
    ]
)

chat_completion_4

<OpenAIObject chat.completion id=chatcmpl-7ynr8ND6rrjvE3Rg4p2vMc0OsPxQm at 0x7f44133b6b70> JSON: {
  "id": "chatcmpl-7ynr8ND6rrjvE3Rg4p2vMc0OsPxQm",
  "object": "chat.completion",
  "created": 1694725682,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\"0ad80e72-02e7-4cfc-a298-cc1568f665ff\": \"'Harder'\", \"c799476d-db52-4dfe-a5f2-187e3894d280\": \"'Easier'\", \"40b468aa-4354-4679-a297-068cc7d8f897\": \"'Harder'\", \"2b6374ec-59d9-4ef2-9d8d-3eb910870b4d\": \"'Same'\", \"8f511d04-bcba-43a6-ac5f-eccdf5f049af\": \"'Easier'\", \"d73751f3-b779-4351-914b-8605dd5c0bfc\": \"32\", \"f73efca0-d7fb-4fb6-9527-884cac8aac41\": \"'Same'\", \"727d4914-85d7-4e5d-8024-0d5af55027ac\": \"'Harder'\", \"1b56e8be-127c-47d3-9864-31fceb586234\": \"'Yes'\"}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 3259,
    "completion_tokens": 250,
    "total_tokens": 3509
  }
}

In [225]:
chat_completion_4_no_eg = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": instruction},
        {"role": "user", "content": "The HTML survey is given below:\n" + shuffled_survey_text},
    ]
)

chat_completion_4_no_eg

<OpenAIObject chat.completion id=chatcmpl-7ynruQC2FDfzIQQvknEYPqJg4yhc0 at 0x7f4412880b90> JSON: {
  "id": "chatcmpl-7ynruQC2FDfzIQQvknEYPqJg4yhc0",
  "object": "chat.completion",
  "created": 1694725730,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n    \"0ad80e72-02e7-4cfc-a298-cc1568f665ff\": \"'Harder'\",\n    \"c799476d-db52-4dfe-a5f2-187e3894d280\": \"'Easier'\",\n    \"40b468aa-4354-4679-a297-068cc7d8f897\": \"'Harder'\",\n    \"2b6374ec-59d9-4ef2-9d8d-3eb910870b4d\": \"'Harder'\",\n    \"8f511d04-bcba-43a6-ac5f-eccdf5f049af\": \"'Harder'\",\n    \"f73efca0-d7fb-4fb6-9527-884cac8aac41\": \"'Same'\",\n    \"727d4914-85d7-4e5d-8024-0d5af55027ac\": \"'Harder'\",\n    \"d73751f3-b779-4351-914b-8605dd5c0bfc\": \"31\",\n    \"1b56e8be-127c-47d3-9864-31fceb586234\": \"'Yes'\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 3076,
    "completion_tokens": 261,


In [235]:
chat_completion_4.choices[0].message.content

'{"0ad80e72-02e7-4cfc-a298-cc1568f665ff": "\'Harder\'", "c799476d-db52-4dfe-a5f2-187e3894d280": "\'Easier\'", "40b468aa-4354-4679-a297-068cc7d8f897": "\'Harder\'", "2b6374ec-59d9-4ef2-9d8d-3eb910870b4d": "\'Same\'", "8f511d04-bcba-43a6-ac5f-eccdf5f049af": "\'Easier\'", "d73751f3-b779-4351-914b-8605dd5c0bfc": "32", "f73efca0-d7fb-4fb6-9527-884cac8aac41": "\'Same\'", "727d4914-85d7-4e5d-8024-0d5af55027ac": "\'Harder\'", "1b56e8be-127c-47d3-9864-31fceb586234": "\'Yes\'"}'

In [236]:
chat_completion_4.to_dict_recursive()

{'id': 'chatcmpl-7ynr8ND6rrjvE3Rg4p2vMc0OsPxQm',
 'object': 'chat.completion',
 'created': 1694725682,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '{"0ad80e72-02e7-4cfc-a298-cc1568f665ff": "\'Harder\'", "c799476d-db52-4dfe-a5f2-187e3894d280": "\'Easier\'", "40b468aa-4354-4679-a297-068cc7d8f897": "\'Harder\'", "2b6374ec-59d9-4ef2-9d8d-3eb910870b4d": "\'Same\'", "8f511d04-bcba-43a6-ac5f-eccdf5f049af": "\'Easier\'", "d73751f3-b779-4351-914b-8605dd5c0bfc": "32", "f73efca0-d7fb-4fb6-9527-884cac8aac41": "\'Same\'", "727d4914-85d7-4e5d-8024-0d5af55027ac": "\'Harder\'", "1b56e8be-127c-47d3-9864-31fceb586234": "\'Yes\'"}'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 3259,
  'completion_tokens': 250,
  'total_tokens': 3509}}